# ASIN to GTIN
## Summary
The goal of this project is to use Python to take in a list of Amazon Standard Identification Numbers(ASINs) and retrieve the products Global Trade Item Number(GTIN) and/or MPN values from Amazon using the Amazon Product Advertising API.

## Import Libraries and Modules

In [1]:
# import libraries
import pandas as pd
import bottlenose # Amazon Product Advertising API Python Library
import re # Regular expression operations module
import time # Time access and conversions module

## Set Credentials for Amazon Product Advertising API
Replaced with values of XXX for security purposes.

In [33]:
AWS_ACCESS_KEY_ID = "XXX"
AWS_SECRET_ACCESS_KEY ="XXX"
AWS_ASSOCIATE_TAG = "XXX"

## Get list of ASINs as pandas DataFrame
List of ASINs is hosted on Google Drive as a CSV file. URL has been shortened for convenience. The path could be changed as needed.

In [1]:
# download list 
!wget -O asin_needs_data.csv 'https://bit.ly/2RM45f5'
print('Download complete!')

--2019-01-23 07:15:53--  https://bit.ly/2RM45f5
Resolving bit.ly... 67.199.248.10, 67.199.248.11
Connecting to bit.ly|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://docs.google.com/spreadsheets/d/e/2PACX-1vSnchQ-ctWClVLy1R8xJhAhCHg7OgMqJCqukvhJNJJBQgbOGCcLDXKSdXeNSxG5XPC6n3DWphYp-bsZ/pub?gid=0&single=true&output=csv [following]
--2019-01-23 07:15:53--  https://docs.google.com/spreadsheets/d/e/2PACX-1vSnchQ-ctWClVLy1R8xJhAhCHg7OgMqJCqukvhJNJJBQgbOGCcLDXKSdXeNSxG5XPC6n3DWphYp-bsZ/pub?gid=0&single=true&output=csv
Resolving docs.google.com... 172.217.2.14
Connecting to docs.google.com|172.217.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘asin_needs_data.csv’

asin_needs_data.csv     [ <=>                ]   1.25K  --.-KB/s    in 0s      

2019-01-23 07:15:54 (25.5 MB/s) - ‘asin_needs_data.csv’ saved [1282]

Download complete!


In [4]:
# create pandas DataFrame with list from CSV file
column_number_with_ASIN = 0 # set this variable to the numberical value of the csv column containing the desired ASINs
asin_df = pd.read_csv('asin_needs_data.csv', header=0, sep=',', usecols=[column_number_with_ASIN]).astype('object')
print(asin_df.head()) # preview DataFrame
print(asin_df.shape) # print DataFrame Shape

         ASIN
0  B004XC6GJ0
1  B000P1DEHU
2  B004NEUJKA
3  B0011ULQNI
4  B001O0DP48
(107, 1)


## Retrieve GTINs from Amazon Product Advertising API

In [5]:
# Constructing a DataFrame for GTIN data by ASIN
asin_data_columns = ['ASIN', 'UPC', 'EAN', 'ISBN']
asin_data_df = pd.DataFrame(columns=asin_data_columns)
asin_data_df

,ASIN,UPC,EAN,ISBN


In [6]:
# function for Amazon AWS Product Advertising API Item Lookup Item Attributes
def aws_gtin(asin):
    amazon = bottlenose.Amazon(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_ASSOCIATE_TAG)
    response = str(amazon.ItemLookup(ItemId=asin, ResponseGroup="ItemAttributes"))
    upc = re.findall('<UPC>(.*)</UPC>', response, flags=0)
    ean = re.findall('<EAN>(.*)</EAN>', response, flags=0)
    isbn = re.findall('<ISBN>(.*)</ISBN>', response, flags=0)
    attrib_list_raw = upc, ean, isbn
    attrib_list = [asin]
    for attrib in attrib_list_raw:
        if len(attrib) > 0:
            attrib_list.append(attrib[0])
        else:
            attrib_list.append('')
    attrib_dict = {'ASIN': attrib_list[0], 'UPC': attrib_list[1], 'EAN' : attrib_list[2], 'ISBN' : attrib_list[3]}
    return(attrib_dict)

In [7]:
# loop to make requests and record responses in DataFrame
for i, asin in enumerate(asin_df['ASIN']):
    print('{} of {}: Getting GTIN data for {}'.format(i + 1, len(asin_df['ASIN']), asin))
    time.sleep(2)
    asin_data_df = asin_data_df.append(aws_gtin(asin), ignore_index=True)

1 of 107: Getting GTIN data for B004XC6GJ0
2 of 107: Getting GTIN data for B000P1DEHU
3 of 107: Getting GTIN data for B004NEUJKA
4 of 107: Getting GTIN data for B0011ULQNI
5 of 107: Getting GTIN data for B001O0DP48
6 of 107: Getting GTIN data for B00DQV2BDO
7 of 107: Getting GTIN data for B000E1B2SO
8 of 107: Getting GTIN data for B00005K3B1
9 of 107: Getting GTIN data for B00000J0E6
10 of 107: Getting GTIN data for B0001B86GI
11 of 107: Getting GTIN data for B075X4G51V
12 of 107: Getting GTIN data for B006N9ZZS4
13 of 107: Getting GTIN data for B003A1LK1O
14 of 107: Getting GTIN data for B00NIAYOX8
15 of 107: Getting GTIN data for B00IO3AQC2
16 of 107: Getting GTIN data for B0028MD7F8
17 of 107: Getting GTIN data for B001DFH0C2
18 of 107: Getting GTIN data for B001DFPR9A
19 of 107: Getting GTIN data for B00A4U1XVQ
20 of 107: Getting GTIN data for B0091HQHZU
21 of 107: Getting GTIN data for B00WLJLYJY
22 of 107: Getting GTIN data for B000IJEZI6
23 of 107: Getting GTIN data for B00PVV54

In [8]:
# preview GTIN data
asin_data_df.head()

,ASIN,UPC,EAN,ISBN
0,B004XC6GJ0,021111531467,0168141495601,
1,B000P1DEHU,027242708952,0027242708952,
2,B004NEUJKA,961613210272,0999993504940,
3,B0011ULQNI,,,
4,B001O0DP48,050633271667,0050633271667,


## Indicate Completeness of Data
It is desireable to have at least one value present for UPC, EAN, or ISBN. A boolean value will be assiged to a new column named "Missing". If the value is True, then the ASIN of that row is lacking UPC, EAN, and ISBN values.

In [13]:
# Construct DataFrame for Missing value
missing_data_columns = ['ASIN','Missing']
asin_data_missing_df = pd.DataFrame(columns = missing_data_columns)
asin_data_missing_df

,ASIN,Missing


In [14]:
# loop to record missing value for each ASIN
missing_data = []
for row in asin_data_df.iterrows():
    missing = True
    if row[1][1] != '':
        missing = False
    elif row[1][2] != '':
        missing = False
    elif row[1][3] != '':
        missing = False
    missing_dict = {'ASIN': row[1][0], 'Missing' : missing}
    asin_data_missing_df = asin_data_missing_df.append(missing_dict, ignore_index=True)

In [15]:
# preview the DataFrame
asin_data_missing_df.head()

,ASIN,Missing
0,B004XC6GJ0,False
1,B000P1DEHU,False
2,B004NEUJKA,False
3,B0011ULQNI,True
4,B001O0DP48,False


In [17]:
# merge DataFrames
asin_data_df = asin_data_df.merge(asin_data_missing_df, on='ASIN')
# preview DataFrame
asin_data_df.head()

,ASIN,UPC,EAN,ISBN,Missing
0,B004XC6GJ0,021111531467,0168141495601,,False
1,B000P1DEHU,027242708952,0027242708952,,False
2,B004NEUJKA,961613210272,0999993504940,,False
3,B0011ULQNI,,,,True
4,B001O0DP48,050633271667,0050633271667,,False


## Results

In [28]:
# Count how many ASINs are missing a GTIN
missing_count = asin_data_df[asin_data_df['Missing'] == True].shape[0]
total_count = asin_data_df.shape[0]
print('Number of ASINs lacking a GTIN: {}'.format(missing_count))
print('Success rate: {}%'.format((total_count - missing_count) / total_count * 100))

Number of ASINs lacking a GTIN: 16
Success rate: 85.04672897196261%


## Append Data to a Tab-separated values file
Tab-separated values are preferable to Comma-separated values when pasting data to most spreadsheet applications. The mode is set to 'a' so that the resulting data is added to existing data rather than overwriting the existing data. This process might be used continually where the TSV file continues to grow.

In [32]:
asin_data_df.to_csv('asin_gtin.tsv', sep='\t' , index=False, mode='a')
print('Export complete.')

Export complete.
